<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Pui/American_Call_jax_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
a = []
while(1):
    a.append('1')

In [27]:
import jax.numpy as jnp
from jax import random
matrix_2d = random.uniform(random.PRNGKey(0), (10, 10))
exclude = [(2, 5), (3, 4), (6, 1)]

ind = tuple(jnp.array(exclude).T)
mask = jnp.ones_like(matrix_2d, dtype=bool).at[ind].set(False)

list_1d = matrix_2d[mask].tolist()
print(mask[:,-1].shape)
print(matrix_2d[-1][mask[:,-1]])
print(mask)

(10,)
[0.08459568 0.04527426 0.21078384 0.4654404  0.7363906  0.23245859
 0.22119188 0.99092937 0.878675   0.4102913 ]
[[ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True False  True  True  True  True]
 [ True  True  True  True False  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True False  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]]


# Test (Skip this if not trying to test, to make sure that functions are defined correctly in cells below without running this cell)

In [ ]:
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

x_sub = jnp.array([[1,1,1,1,1],
                   [1.08,1.07,0.97,0.77,0.84],
                   [1.08**2,1.07**2,0.97**2,0.77**2,0.84**2]]).transpose()
y_sub = jnp.array([0, 0.07*0.94176, 0.18*0.94176, 0.2*0.94176, 0.09*0.94176])

print(jax.numpy.linalg.lstsq(x_sub, y_sub)[0])
print(x_sub.dot(jax.numpy.linalg.lstsq(x_sub, y_sub)[0]))

[-1.0699916  2.9834177 -1.8135799]
[0.03674006 0.04589796 0.11752641 0.1519686  0.15641725]


In [ ]:
import numpy as np
x_sub = np.array([[1,1,1,1,1],
                   [1.08,1.07,0.97,0.77,0.84],
                   [1.08**2,1.07**2,0.97**2,0.77**2,0.84**2]]).transpose()
y_sub = np.array([0, 0.07*0.94176, 0.18*0.94176, 0.2*0.94176, 0.09*0.94176])

print(np.linalg.lstsq(x_sub, y_sub, rcond=None)[0])
print(x_sub.dot(np.linalg.lstsq(x_sub, y_sub)[0]))

[-1.0699825   2.98339626 -1.81356745]
[0.03674038 0.04589812 0.11752626 0.15196848 0.15641716]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [ ]:
X = jnp.array([92.8,96.34,89.37,86.12])
X_2 = jnp.c_[np.ones_like(X), X, X**2]
Y = jnp.array([0,4.1759,6.7863,6.9345])

print(jax.numpy.linalg.lstsq(X_2, Y, rcond=-1)[0])
print(X_2.dot(jax.numpy.linalg.lstsq(X_2, Y, rcond = -1)[0]))

[ 8.8335626e+02 -1.8860060e+01  1.0095285e-01]
[2.5325317 3.3614502 4.1428833 7.8602905]


In [ ]:
X = np.array([92.8,96.34,89.37,86.12])
X_2 = np.c_[np.ones_like(X), X, X**2]
Y = np.array([0,4.1759,6.7863,6.9345])

print(np.linalg.lstsq(X_2, Y, rcond=None)[0])
print(X_2.dot(np.linalg.lstsq(X_2, Y, rcond=None)[0]))

[ 8.83369607e+02 -1.88603524e+01  1.00954440e-01]
[2.53238803 3.3613511  4.1427116  7.86024927]


# Change to CF Array

In [ ]:
# change from CF matrix to CF array
# call
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  print('out')
  print(out)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  print('out_avg')
  print(out_avg)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  #print(ITM_matrix)
  CF_array = jnp.maximum(out_avg[:,-1]-K, 0) # array that will store discounted cash flows for the next time period
  print('CF_array')
  print(CF_array)

  print('ITM_matrix')
  print(ITM_matrix)
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    print(i)
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    print('X')
    print(X)
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
    print('Y')
    print(Y)
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value
    print('ECV')
    print(ECV)

    print(jnp.array(ITM_index[jnp.where(X-K > ECV)[0]]))
    # update CF_array
    CF_array = jax.ops.index_update(x=CF_array, 
                                    idx=jax.ops.index[jnp.array(ITM_index[jnp.where(X-K > ECV)[0]])],  # index subsetting paths that has X-K > ECV
                                    y=(X-K)[jnp.where(X-K > ECV)[0]]) # X-K if X-K > ECV, else 0
    print('CF_array')
    print(CF_array)

  return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 5

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 6
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 80.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

out
[[[ 96.21578 ]
  [ 91.14775 ]
  [100.194565]
  [ 98.64239 ]
  [100.26974 ]
  [110.80411 ]]

 [[ 76.17836 ]
  [ 88.16234 ]
  [ 89.02843 ]
  [ 88.396324]
  [ 76.27098 ]
  [ 62.58006 ]]

 [[ 96.975716]
  [ 85.91724 ]
  [ 84.55148 ]
  [ 88.01227 ]
  [ 78.10432 ]
  [ 75.73002 ]]

 [[100.92803 ]
  [105.0573  ]
  [116.7805  ]
  [146.18767 ]
  [152.94846 ]
  [128.80319 ]]

 [[ 94.14551 ]
  [ 88.50854 ]
  [ 73.177025]
  [ 71.277466]
  [ 65.9051  ]
  [ 75.834854]]]
out_avg
[[ 96.21578   91.14775  100.194565  98.64239  100.26974  110.80411 ]
 [ 76.17836   88.16234   89.02843   88.396324  76.27098   62.58006 ]
 [ 96.975716  85.91724   84.55148   88.01227   78.10432   75.73002 ]
 [100.92803  105.0573   116.7805   146.18767  152.94846  128.80319 ]
 [ 94.14551   88.50854   73.177025  71.277466  65.9051    75.834854]]
CF_array
[30.804108  0.        0.       48.803192  0.      ]
ITM_matrix
[[ True  True  True  True  True  True]
 [False  True  True  True False False]
 [ True  True  True  True False 

In [ ]:
# change from CF matrix to CF array
# call
# r is constant here for testing
# large numpaths & numsteps to test price here

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(out_avg[:,-1]-K, 0) # array that will store discounted cash flows for the next time period
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value

    # update CF_array
    CF_array = jax.ops.index_update(x=CF_array, 
                                    idx=jax.ops.index[jnp.array(ITM_index[jnp.where(X-K > ECV)[0]])],  # index subsetting paths that has X-K > ECV
                                    y=(X-K)[jnp.where(X-K > ECV)[0]]) # X-K for these paths

  return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 1000
numsteps = 100

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 90.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# delta test
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

14.875802
[0.75861514]


In [ ]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# test speed

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack
import time

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  # tic = time.perf_counter()
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  # toc = time.perf_counter()
  # print(f"out: {toc - tic:0.7f} seconds")

  # tic = time.perf_counter()
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  # toc = time.perf_counter()
  # print(f"out_avg: {toc - tic:0.7f} seconds")

  dt = T / numsteps

  # tic = time.perf_counter()
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  # toc = time.perf_counter()
  # print(f"ITM_matrix Initialization: {toc - tic:0.7f} seconds")

  # tic = time.perf_counter()
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period
  # toc = time.perf_counter()
  # print(f"CF_array Initialization: {toc - tic:0.7f} seconds")
 
  # # update t=T-1 to t=0
  # for i in range(2, numsteps+1):
  #   tic = time.perf_counter()
  #   ITM_index = jnp.where(ITM_matrix[:,-i])[0]
  #   toc = time.perf_counter()
  #   print(f"ITM_index: {toc - tic:0.7f} seconds")

  #   tic = time.perf_counter()
  #   X = out_avg[ITM_index,-i]
  #   toc = time.perf_counter()
  #   print(f"X: {toc - tic:0.7f} seconds")

  #   tic = time.perf_counter()
  #   CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
  #   toc = time.perf_counter()
  #   print(f"CF_array discount: {toc - tic:0.7f} seconds")

  #   tic = time.perf_counter()
  #   Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
  #   toc = time.perf_counter()
  #   print(f"Y: {toc - tic:0.7f} seconds")

  #   tic = time.perf_counter()
  #   X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
  #   toc = time.perf_counter()
  #   print(f"X_matrix: {toc - tic:0.7f} seconds")

  #   tic = time.perf_counter()
  #   ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value
  #   toc = time.perf_counter()
  #   print(f"ECV lstsq: {toc - tic:0.7f} seconds")

  #   tic = time.perf_counter()
  #   # update CF_array
  #   CF_array = jax.ops.index_update(x=CF_array, 
  #                                   idx=jax.ops.index[jnp.array(ITM_index[jnp.where(K-X > ECV)[0]])],  # index subsetting paths that has K-X > ECV
  #                                   y=(K-X)[jnp.where(K-X > ECV)[0]]) # K-X for these paths
  #   toc = time.perf_counter()
  #   print(f"CF_array update: {toc - tic:0.7f} seconds")

  # return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

  def for_loop(key2, numsteps, r, K, dt, ITM_matrix, CF_array):
    def body_fun(i, val):
      #ITM_index = jnp.where(ITM_matrix[:,-i], size = ITM_matrix[:,-i].sum())[0]
      X = out_avg[:,-i]
      val = (val * jnp.exp(-r * dt)) # discount CF_array by one step
      Y = val # Y is subset of CF_array that is ITM
      X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
      ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value

      # update CF_array
      val = jnp.maximum(K-X, ECV) # K-X for these paths
      return val
    CF_array =  jax.lax.fori_loop(2, numsteps+1, body_fun, CF_array)[1:]
    return (CF_array * jnp.exp(-r * dt)).mean()

  np.random.seed(np.random.randint(10000))
  rng = jax.random.PRNGKey(np.random.randint(10000))
  rng, key2 = jax.random.split(rng)
  fast_simple = jax.jit(for_loop, static_argnums=2)

  CF_out = fast_simple(key2, numsteps, r.copy()[0], K, dt, ITM_matrix, CF_array)
  return CF_out

numstocks = 1
numpaths = 50000
numsteps = 50

np.random.seed(np.random.randint(10000))

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)
drift = jnp.array([0.02]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.2*0.2
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 100.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.3384

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5610
####################################################################################

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# # delta test
# goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
# print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

7.4203873


In [ ]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# large numpaths & numsteps to test price here
# replace for loop with jax.lax.fori_loop -> faster
# but couldn't do grad using fori_loop

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period
 
  def for_loop(key2, numsteps, r, K, dt, ITM_matrix, CF_array):
    def body_fun(i, val):
      X = out_avg[:,-i]
      val = (val * jnp.exp(-r * dt)) # discount CF_array by one step
      Y = val # Y is CF_array
      X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
      ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value

      # update CF_array
      KX_larger_than_ECV = (K-X > ECV)
      val = KX_larger_than_ECV*(K-X) + (1-KX_larger_than_ECV)*val
      return val

    CF_array =  jax.lax.fori_loop(2, numsteps+1, body_fun, CF_array)[1:]
    return (CF_array * jnp.exp(-r * dt)).mean()

  np.random.seed(np.random.randint(10000))
  rng = jax.random.PRNGKey(np.random.randint(10000))
  rng, key2 = jax.random.split(rng)
  fast_simple = jax.jit(for_loop, static_argnums=2)

  CF_out = fast_simple(key2, numsteps, r.copy()[0], K, dt, ITM_matrix, CF_array)
  return CF_out

numstocks = 1
numpaths = 10000
numsteps = 500

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)
drift = jnp.array([0.05]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([200.]*numstocks) # must be float

T = 1.0
K = 198.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.3384

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5610
####################################################################################

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
%timeit(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# delta test
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

14.604304
1 loop, best of 5: 526 ms per loop


ValueError: ignored

In [ ]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# large numpaths & numsteps to test price here
# replace for loop with jax.lax.fori_loop w/ vmap -> faster
# but couldn't do grad using fori_loop

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def for_loop(key2, numsteps, r, K, dt, ITM_matrix, out_avg, CF_array):
    def body_fun(i, val):
      X = out_avg[:,-i]
      val = (val * jnp.exp(-r * dt)) # discount CF_array by one step
      Y = val # Y is CF_array
      X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
      ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value

      # update CF_array
      KX_larger_than_ECV = (K-X > ECV)
      val = KX_larger_than_ECV*(K-X) + (1-KX_larger_than_ECV)*val
      return val

    CF_array =  jax.lax.fori_loop(2, numsteps+1, body_fun, CF_array)[1:]
    return (CF_array * jnp.exp(-r * dt)).mean()

def optionvalueavg(key, key2, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period  

  CF_out = CF_array_compile_vmap(keys2, numsteps, r.copy()[0], K, dt, ITM_matrix, out_avg, CF_array)
  return CF_out[0]

numstocks = 1
numpaths = 10000
numsteps = 500

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)
drift = jnp.array([0.05]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([200.]*numstocks) # must be float

T = 1.0
K = 198.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

rng2 = jax.random.PRNGKey(np.random.randint(10000))
rng2, key2 = jax.random.split(rng)

CF_array_compile = jax.jit(for_loop, static_argnums=2)
keys2 = jax.random.split(key2, 1)
CF_array_compile_vmap = jax.vmap(CF_array_compile, in_axes=(0, None, None, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.3384

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5610
####################################################################################

# option price
print(optionvalueavg(key, key2, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
%timeit(optionvalueavg(key, key2, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# delta test
goptionvalueavg = jax.grad(optionvalueavg,argnums=2)
print(goptionvalueavg(keys, key2, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

14.49698
10 loops, best of 5: 176 ms per loop


ValueError: ignored

In [ ]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# large numpaths & numsteps to test price here
# use normal for loop

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period
 
  # # update t=T-1 to t=0
  # for i in range(2, numsteps+1):
  #   ITM_index = jnp.where(ITM_matrix[:,-i])[0]
  #   X = out_avg[ITM_index,-i]
  #   CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
  #   Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
  #   X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
  #   #ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
  #   ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value

  #   # update CF_array
  #   CF_array = jax.ops.index_update(x=CF_array, 
  #                                   idx=jax.ops.index[jnp.array(ITM_index[jnp.where(K-X > ECV)[0]])],  # index subsetting paths that has K-X > ECV
  #                                   y=(K-X)[jnp.where(K-X > ECV)[0]]) # K-X for these paths

  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    X = out_avg[:,-i]
    CF_array = (CF_array * jnp.exp(-r * dt)) # discount CF_array by one step
    #Y = CF_array # Y is CF_array
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, CF_array, rcond=-1)[0]) # predicted Y - ECV - expected continue value

    KX_larger_than_ECV = (K-X > ECV)
    CF_array = KX_larger_than_ECV*(K-X) + (1-KX_larger_than_ECV)*CF_array

  return  (CF_array * jnp.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 10000
numsteps = 1000

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)
drift = jnp.array([0.05]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([1.]*numstocks) # must be float

T = 1.0
K = 1.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.3384

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5610
####################################################################################

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
%timeit(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# delta test
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
%timeit(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

0.079304814
1 loop, best of 5: 9.01 s per loop
[-0.44878355]
1 loop, best of 5: 44 s per loop


In [82]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# large numpaths & numsteps to test price here
# use normal for loop
# test jnp.where

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period

  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_boolean = ITM_matrix[:,-i]
    print('ITM_boolean')
    print(ITM_boolean)
    X = jnp.where(ITM_boolean, out_avg[:,-i], jnp.nan) # if ITM, use out_avg, if not, use nan
    print('X')
    print(X)
    CF_array = (CF_array * jnp.exp(-r * dt)) # discount CF_array by one step
    print('CF_array')
    print(CF_array)
    Y = jnp.where(ITM_boolean, CF_array, jnp.nan) # if ITM, Y = CF array, if not, use nan
    print('Y')
    print(Y)
    X_matrix = jnp.array([jnp.where(ITM_boolean, X**0, jnp.nan),
                          X**1,
                          X**2]).transpose()
    print('X_matrix')
    print(X_matrix)
    print(X_matrix[~jnp.isnan(X_matrix).any(axis=1),:])
    print(Y[~jnp.isnan(Y)])

    print('lstsq')
    print(jax.numpy.linalg.lstsq(X_matrix[~jnp.isnan(X_matrix).any(axis=1),:], Y[~jnp.isnan(Y)], rcond=-1)[0])

    print('formula')

    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value
    print('ECV')
    print(ECV)

    KX_larger_than_ECV = (K-X > ECV)
    CF_array = KX_larger_than_ECV*(K-X) + (1-KX_larger_than_ECV)*CF_array

  return  (CF_array * jnp.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 5
numsteps = 2

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)
drift = jnp.array([0.05]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([1.]*numstocks) # must be float

T = 1.0
K = 1.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.3384

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5610
####################################################################################

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
# %timeit(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# # delta test
# goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
# print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
# %timeit(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

ITM_boolean
[ True  True  True False False]
X
[0.8514236 0.9754792 0.9290242       nan       nan]
CF_array
[0.33547902 0.         0.18662047 0.         0.19267713]
Y
[0.33547902 0.         0.18662047        nan        nan]
X_matrix
[[1.         0.8514236  0.7249222 ]
 [1.         0.9754792  0.95155966]
 [1.         0.9290242  0.863086  ]
 [       nan        nan        nan]
 [       nan        nan        nan]]
[[1.         0.8514236  0.7249222 ]
 [1.         0.9754792  0.95155966]
 [1.         0.9290242  0.863086  ]]
[0.33547902 0.         0.18662047]
[-11.41395  28.20486 -16.91886]
ECV
[nan nan nan nan nan]
0.13942574


In [ ]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# large numpaths & numsteps to test price here
# use normal for loop
# test X and X_full -> too slow

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    print(ITM_index)
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    #print(CF_array)

    X = out_avg[ITM_index,-i]
    #print(X)
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    rg_coeff = jax.numpy.linalg.lstsq(X_matrix, CF_array[ITM_index], rcond=-1)[0]
    #print(rg_coeff)

    X_full = out_avg[:,-i]
    #print(X_full)
    X_matrix_full = jnp.array([X_full**0,X_full**1,X_full**2]).transpose()
    ECV = X_matrix_full.dot(rg_coeff) # predicted Y - ECV - expected continue value
    #print(ECV)

    #print(K-X_full)
    #print(K-X_full > ECV)

    Exercise = jnp.where(ITM_matrix[:,-i], K-X_full > ECV, False)
    #print(Exercise)
    CF_array = Exercise*(K-X_full) + (1-Exercise)*CF_array

  return  (CF_array * jnp.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 5000
numsteps = 50

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)
drift = jnp.array([0.05]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([1.]*numstocks) # must be float

T = 1.0
K = 1.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.3384

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5610
####################################################################################

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
# %timeit(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# # delta test
# goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
# print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
# %timeit(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

In [92]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# large numpaths & numsteps to test price here
# replace jax.lax.fori_loop with jax.lax.scan -> can do grad now

import jax
import jax.numpy as jnp
from jax import random
from jax import jit, lax
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period

  def for_loop(numsteps, r, K, dt, ITM_matrix, CF_array):
    def body_fun(val, i):
      X = out_avg[:,-i]
      val = (val * jnp.exp(-r * dt)) # discount CF_array by one step
      Y = val # Y is CF_array
      X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
      ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value

      # update CF_array
      KX_larger_than_ECV = (K-X > ECV)
      val = KX_larger_than_ECV*(K-X) + (1-KX_larger_than_ECV)*val
      return val, i

    counts = jnp.arange(2, numsteps+1)
    CF_array, _ =  jax.lax.scan(body_fun, CF_array, counts)
    return (CF_array * jnp.exp(-r * dt)).mean()

  # def for_loop(numsteps, r, K, dt, ITM_matrix, CF_array):
  #   def body_fun(val, i):
  #     ITM_index = jnp.where(ITM_matrix[:,-i])[0]
  #     X = out_avg[ITM_index,-i]
  #     val = (val * np.exp(-r * dt)) # discount CF_array by one step
  #     Y = val[ITM_index] # Y is subset of CF_array that is ITM
  #     X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
  #     #ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
  #     ECV = X_matrix.dot(jnp.array(np.linalg.lstsq(X_matrix, Y, rcond=None)[0])) # predicted Y - ECV - expected continue value

  #     # update CF_array
  #     val = jax.ops.index_update(x=val, 
  #                                     idx=jax.ops.index[jnp.array(ITM_index[jnp.where(K-X > ECV)[0]])],  # index subsetting paths that has K-X > ECV
  #                                     y=(K-X)[jnp.where(K-X > ECV)[0]]) # K-X for these paths
  #     return val, i

  #   counts = jnp.arange(2, numsteps+1)
  #   CF_array, _ =  jax.lax.scan(body_fun, CF_array, counts)
  #   return (CF_array * jnp.exp(-r * dt)).mean()

  CF_out = for_loop(numsteps, r.copy()[0], K, dt, ITM_matrix, CF_array)
  return CF_out

numstocks = 1
numpaths = 10000
numsteps = 1000

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)
drift = jnp.array([0.05]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([1.]*numstocks) # must be float

T = 1.0
K = 1.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.2(1stock) 0.34641016151(3stock), 1
# option price should be around 2.33
# np price: 2.3384

#S, K, r, drift, sigma, T
# 100, 100, 0.01, 0.01, 0.25, 1
# option price should be around 9.51
# np price: 9.5610
####################################################################################

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
# %timeit(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# delta test
# goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
# print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
# %timeit(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

0.07976446


In [54]:
X = jnp.array([[1,2,3,4,5,6],[7,8,9,10,11,12]])
mask = jnp.array([[True, True, False, True, False, True],[True, False, False, True, False, True]])

X[mask]

DeviceArray([ 1,  2,  4,  6,  7, 10, 12], dtype=int32)

In [65]:
ITM_boolean = mask[:,-2]
jnp.where(1-ITM_boolean, X[:,-2], jnp.nan)

DeviceArray([ 5., 11.], dtype=float32, weak_type=True)

In [52]:
thing = np.array([-1,0,1,2,3])
n_valid_things = np.sum(thing > 0)
print(n_valid_things)
valid_thing_indices = np.where(thing > 0, np.arange(thing.shape[0]), thing.shape[0])
print(valid_thing_indices)
padded_things = np.concatenate([thing, np.array([0])])
print(padded_things)
padded_valid_things = padded_things[np.sort(valid_thing_indices)]
print(padded_valid_things)

3
[5 5 2 3 4]
[-1  0  1  2  3  0]
[1 2 3 0 0]


# Construct Neural Net

In [ ]:
%%writefile cupy_dataset.py
import cupy
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, keys): # need to pass 'keys'
    return jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:], axis=1)-K,0)) * jnp.exp(-r[0] * T))
    # must use '-1' not 'numsteps', or else grad will be 0

###################################################################################################
# these 2 functions must be defined outside class in order to be used in 'optionvalueavg' function
fast_simple = jax.jit(Brownian_motion, static_argnums=2)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))
###################################################################################################

class OptionDataSet(object):
    
    def __init__(self, max_len, number_path, batch, seed, stocks):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 50
        self.N_BATCH = batch
        self.N_STOCKS = stocks
        self.T = 1.0 # assume T = 1, use float here
        self.seed = seed
        np.random.seed(seed)
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num >= self.max_length:
            raise StopIteration
        
        Y = cupy.zeros((self.N_BATCH, 1 + self.N_STOCKS), dtype=cupy.float32) # output: price, delta1, delta2, delta3
        X = cupy.zeros((self.N_BATCH, self.N_STOCKS * 6), dtype = cupy.float32)

        for op in range(self.N_BATCH):
          
          rng = jax.random.PRNGKey(self.seed)
          rng, key = jax.random.split(rng)

          ################################################################################################### generate random input numbers

          initial_stocks = jnp.array(np.random.random(self.N_STOCKS) * 200.0)

          corr = jnp.diag(jnp.array([1]*self.N_STOCKS)) # assume no correlation between stocks here
          sigma = jnp.array(np.random.random(self.N_STOCKS) * 0.4)
          cov = (jnp.diag(sigma)).dot(corr).dot(jnp.diag(sigma))

          r = jnp.repeat(jnp.array(np.random.random(1) * 0.1), self.N_STOCKS)
          drift = r # To match BS, use drift = r

          T = self.T
          K = np.random.random(1) * 200.0

          ###################################################################################################
          ################################################################################################### apply functions to compute price and deltas
          
          keys = jax.random.split(key, self.N_PATHS)

          European_Call_price = optionvalueavg(key, initial_stocks, self.N_STEPS, drift, r, cov, K, T, keys) # need to pass 'keys'
          gooptionvalue = jax.grad(optionvalueavg, argnums=1)
          Deltas = gooptionvalue(keys, initial_stocks, self.N_STEPS, drift, r, cov, K, T, keys) # need to pass 'keys'

          ###################################################################################################
          ################################################################################################### store input and output numbers in X and Y

          Y[op, 0] = European_Call_price
          Y[op, 1:4] = cupy.array(Deltas, dtype=cupy.float32)

          paras = (jnp.repeat(jnp.array(T), self.N_STOCKS), jnp.repeat(jnp.array(K), self.N_STOCKS), initial_stocks, sigma, drift, r)
          paras = np.column_stack(paras).reshape(1,-1)[0]
          X[op,] = cupy.array(paras)

          ###################################################################################################

        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))


# ds = OptionDataSet(max_len = 2, number_path = 10000, batch = 2, seed = 15, stocks=3) # for testing purpose, use constant seed. When training, change to random seed
# for i in ds:
#     print(i)

Writing cupy_dataset.py


In [ ]:
%%writefile model.py
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np

class Net(nn.Module):

    def __init__(self, hidden=1024):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6*3, hidden) # remember to change this!
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, hidden)
        self.fc5 = nn.Linear(hidden, hidden)
        self.fc6 = nn.Linear(hidden, 4) # 4 outputs: price, delta1, delta2, delta3
        self.register_buffer('norm',
                             torch.tensor([1, 200.0, 200.0, 0.4, 0.1, 0.1]*3)) # don't use numpy here - will give error later

    def forward(self, x):
        # normalize the parameter to range [0-1] 
        x = x / self.norm
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.elu(self.fc4(x))
        x = F.elu(self.fc5(x))
        return self.fc6(x)

Writing model.py


# Train Neural Net

In [ ]:
!pip install pytorch-ignite

     |████████████████████████████████| 232 kB 10.0 MB/s 


In [ ]:
# If memory is not enough, try changing parameters and restarting session
# loss will converge

from ignite.engine import Engine, Events
from ignite.handlers import Timer
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import OptionDataSet
import numpy as np
timer = Timer(average=True)
model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
dataset = OptionDataSet(max_len = 100, number_path = 1024, batch = 32, seed = np.random.randint(10000), stocks = 3) # must have random seed


def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    #print(y)
    y_pred = model(x)
    #print(y_pred)
    loss = loss_fn(y_pred[:,:], y[:,:]) # compute MSE between the 2 arrays
    #print(loss)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 20

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value(), 'iter num', iter)
        
trainer.run(dataset, max_epochs = 100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: /usr/local/lib/python3.7/dist-packages/ignite/contrib/handlers/param_scheduler.py has been moved to /ignite/handlers/param_scheduler.py and will be removed in version 0.6.0.
 Please refer to the documentation for more details.
  """


loss 308.3535461425781 average time 0.5890158134499984 iter num 20
loss 309.395263671875 average time 0.29650749102499674 iter num 40
loss 319.1919250488281 average time 0.1990294852333278 iter num 60
loss 232.41702270507812 average time 0.15041632617499587 iter num 80
loss 169.25584411621094 average time 0.12112373191999723 iter num 100
loss 41.82872009277344 average time 0.0727944900500006 iter num 20
loss 8.219586372375488 average time 0.038342809124999634 iter num 40
loss 2.8115668296813965 average time 0.026894211649999042 iter num 60
loss 2.778562545776367 average time 0.021186888737497613 iter num 80
loss 2.3859236240386963 average time 0.017774565589996884 iter num 100
loss 1.3037325143814087 average time 0.07461170490000768 iter num 20
loss 1.4311182498931885 average time 0.039529715224995245 iter num 40
loss 0.608491837978363 average time 0.027656891716664706 iter num 60
loss 0.5415693521499634 average time 0.02177082573749658 iter num 80
loss 0.3868729770183563 average time 

ERROR:ignite.engine.engine.Engine:Engine run is terminating due to exception: 


KeyboardInterrupt: ignored

**Save Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
model_save_name = 'jax_european_test_2.pth'
path = F"/content/drive/MyDrive/AFP Project/PUI/{model_save_name}" 
torch.save(model.state_dict(), path)

**Load Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
model_save_name = 'Sobolev_test_2.pth'
path = F"/content/drive/MyDrive/AFP Project/PUI/{model_save_name}" 
state_dict = torch.load(path)
print(state_dict.keys())

odict_keys(['norm', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias', 'fc5.weight', 'fc5.bias', 'fc6.weight', 'fc6.bias'])


In [ ]:
# need to run 'Writing cupy_dataset.py' and 'Writing model.py' above before this
from model import Net
model = Net().cuda()

model.load_state_dict(state_dict)
print(model)

Net(
  (fc1): Linear(in_features=18, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=1024, bias=True)
  (fc5): Linear(in_features=1024, out_features=1024, bias=True)
  (fc6): Linear(in_features=1024, out_features=7, bias=True)
)


**Continue to train model**

In [ ]:
# If memory is not enough, try changing parameters and restarting session
# loss will converge

from ignite.engine import Engine, Events
from ignite.handlers import Timer
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import OptionDataSet
import numpy as np
timer = Timer(average=True)
#model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
#dataset = OptionDataSet(max_len = 100, number_path = 1024, batch = 32, seed = np.random.randint(10000), stocks = 3) # must have random seed
dataset = OptionDataSet(max_len = 100, number_path = 10000, batch = 8, seed = np.random.randint(10000), stocks = 3) # must have random seed


def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    #print(y)
    y_pred = model(x)
    #print(y_pred)
    loss = loss_fn(y_pred[:,:], y[:,:]) # compute MSE between the 2 arrays
    #print(loss)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 20

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value(), 'iter num', iter)
        
trainer.run(dataset, max_epochs = 10)

model_save_name = 'jax_european_test_3.pth'
path = F"/content/drive/MyDrive/AFP Project/PUI/{model_save_name}" 
torch.save(model.state_dict(), path)

loss 3.847933769226074 average time 0.024525942649995615 iter num 20
loss 0.11467143893241882 average time 0.014268842749987697 iter num 40
loss 0.7498358488082886 average time 0.010797686849988017 iter num 60
loss 0.5012774467468262 average time 0.00905447164998634 iter num 80
loss 0.08808335661888123 average time 0.008048051719988507 iter num 100
loss 0.5424646139144897 average time 0.023962988050016065 iter num 20
loss 0.09199616312980652 average time 0.013907441924999375 iter num 40
loss 0.41971147060394287 average time 0.010556156083328005 iter num 60
loss 0.9499955773353577 average time 0.008943276199994443 iter num 80
loss 0.039512861520051956 average time 0.007942020249997768 iter num 100
loss 0.038578908890485764 average time 0.023953661050018126 iter num 20
loss 0.7367726564407349 average time 0.013887499250017754 iter num 40
loss 0.3312565088272095 average time 0.010584145183342268 iter num 60
loss 0.16353926062583923 average time 0.008966675425003245 iter num 80
loss 0.0240

#Results

In [ ]:
import torch
inputs = torch.tensor([[1, 110.0, 100.0, 0.25, 0., 0.]*3]).cuda()
model(inputs.float())

# price, delta1, delta2, delta3
# should be around (2.3654, 0.0937, 0.0937, 0.0937)

tensor([[2.3494, 0.0840, 0.0863, 0.0899]], device='cuda:0',
       grad_fn=<AddmmBackward>)